In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import scipy.stats as st
from datetime import datetime
from datetime import date
from pandas.tseries.holiday import USFederalHolidayCalendar
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

train = pd.read_csv("../input/train.csv", nrows = 10009000)
print(train.shape)
# Any results you write to the current directory are saved as output.
import lightgbm as lgbm

In [ ]:
print(train.shape)

In [ ]:
import gc
gc.collect()

In [ ]:
# train[(train['fare_amount']>40) & (train['fare_amount']<100)
#      ].sort_values(by = 'fare_amount', ascending = False).head(n = 20)

In [ ]:
# display(train1['fare_amount'].describe())
# sns.distplot(train1['fare_amount'])

## Train using manhattan distance between pickup and dropoff point.

In [ ]:
imp_locations = [(40.639722, -73.778889), (40.6925, -74.168611),
                (40.77725, -73.872611)]

In [ ]:
cal = USFederalHolidayCalendar()
holidays = pd.to_datetime(cal.holidays(start='2009-01-01', end='2015-12-31'))

def haverSine(df):
    Radius = 6371
    h = np.sqrt((np.sin(df['abs_latitude_diff']/2)*
                       np.sin(df['abs_latitude_diff']/2))+
                       (np.cos(df['pickup_latitude'])*
                        np.cos(df['dropoff_latitude'])
                       *(np.sin(df['abs_longitude_diff']/2)*
                        np.sin(df['abs_longitude_diff']/2))))
    df['haversine'] = 2*Radius*np.arcsin(h)
    
def calcHaversine(lat1, long1, lat2, long2):
    Radius = 6371
    h = np.sqrt((np.sin((lat1 - lat2)/2)*
                       np.sin((lat1- lat2)/2))+
                       (np.cos(lat1)*
                        np.cos(lat2)
                       *(np.sin((long1 - long2)/2)*
                        np.sin((long1 - long2)/2))))
    return 2*np.arcsin(h)
# import math
# def haverSine(df):
#     pi = math.pi
#     Radius = 6371.0
#     h = ((np.sin(((pi/180.0)*df['abs_latitude_diff'])/2)**2)+
#                         (np.cos((pi/180.0)*df['pickup_latitude'])*
#                         np.cos((pi/180.0)*df['dropoff_latitude'])
#                        *(np.sin((pi/180.0)*df['abs_longitude_diff']/2)**2)))
#     df['haversine'] = 2*Radius*np.arctan2(np.sqrt(h), np.sqrt(1-h))
    
def getDateTime(df):
    df['datetime'] = pd.to_datetime(df['pickup_datetime'], 
                                    format = "%Y-%m-%d %H:%M:%S %Z")
    
def getYear(df):
    df['year'] = df['datetime'].dt.year
    
def getTime(df):
    df['time'] = df['datetime'].dt.hour
    
def getMonth(df):
    df['month'] = df['datetime'].dt.month
    
def getWeekday(df):
    df['weekday'] = df['datetime'].dt.weekday
    
#D(0) if Day, N(1) if night, 
def getDayOrNight(df):
    df['light'] = df['time'].apply(lambda x:"D" if (x >5 and x <18)
                                  else "A" if(x >=18 and x<=23) else "N")
def getLightEncoding(df):
    df['light_d'] = df['light'].apply(lambda x:1 if x == "D" else 0)
    df['light_a'] = df['light'].apply(lambda x:1 if x == "N" else 0)
    df['light_n'] = df['light'].apply(lambda x:1 if x == "a" else 0)
    
def distFromImpLocations(df):
    for i in range(len(imp_locations)):
        df['imp_'+str(i)+"a"] = calcHaversine(df.dropoff_latitude,
                                              df.dropoff_longitude,
                                          imp_locations[i][0], imp_locations[i][1])
        df['imp_'+str(i)+"b"] = calcHaversine(df.dropoff_latitude, 
                                      df.dropoff_longitude,
                                  imp_locations[i][0], imp_locations[i][1])
        
def isHoliday(df):
    df['isHoliday'] = df['datetime'].apply(lambda x:1 if x.date() in holidays
                                          else 0)
    
#         df['imp_'+str(i)+"b"] = calcHaversine(df.dropoff_latitude, 
#                                               df.dropoff_longitude,
#                                           imp_locations[i][0], imp_locations[i][1])

In [ ]:
train['abs_longitude_diff'] = (train['pickup_longitude'] - 
                               train['dropoff_longitude']).abs()
train['abs_latitude_diff'] = (train['pickup_latitude'] - 
                                 train['dropoff_latitude']).abs()
haverSine(train)
getDateTime(train)
getYear(train)
getMonth(train)
getTime(train)
getWeekday(train)
getDayOrNight(train)
distFromImpLocations(train)
isHoliday(train)

### Remove null Values

In [ ]:
#Remove null values
train.isnull().sum()

In [ ]:
#drop the rows having null values
print("OldSize = ", train.shape)
train.dropna(axis = 'rows', how = 'any', inplace = True)
print("newSize =" , train.shape)

In [ ]:
# f = pd.melt(train, id_vars = ['fare_amount'], value_vars = ['abs_longitude_diff'
#                                                          ,'abs-latitude_diff'])
# g = sns.FacetGrid(f, col = "variable", col_wrap = 2, sharex = False, 
#                   sharey = False, size = 4)
# g.map(sns.regplot, "value", "fare_amount", fit_reg = False)

## Fill Null Values

In [ ]:
train.loc[train['passenger_count'] ==0 ,'passenger_count'
         ]= train['passenger_count'].mode()[0]
train.loc[train['haversine'] == 0, 'haversine'
         ] = train['haversine'].mean()

### ------------Start from here to use different model------------

In [ ]:
#Remove Outliers
print("OldSize = ", train.shape)
train1 = train.loc[(train.abs_longitude_diff < 5.0) &
              (train['abs_latitude_diff'] < 5.0) &
              (train['fare_amount'] > 0.0)&
              (train.fare_amount<100.0)&
              (train.pickup_longitude<=-71.0) & 
               (train.pickup_longitude>=-75)&
               (train.pickup_latitude>=40.0)&
              (train.pickup_latitude<=42.0)&
               (train.dropoff_longitude>=-75.0)&
               (train.dropoff_longitude<=-71.0)&
                (train.dropoff_latitude<=42.0)&
               (train.dropoff_latitude>=40.0),:]
# (train['passenger_count']>0.00)
print("NewSize = ",train1.shape)

In [ ]:
#Get Encoding for time colums
getLightEncoding(train1)

### Prepare Data For Training*

In [ ]:
train2 = np.column_stack((train1['abs_longitude_diff'], 
                           train1['abs_longitude_diff']**2,
                          train1['abs_latitude_diff'],
                           train1['abs_latitude_diff']**2,
                           train1.haversine,
                           train1['haversine']**2,
                          train1.pickup_longitude,
                          train1.pickup_latitude,
                          train1.dropoff_longitude,
                          train1.dropoff_latitude,
                          train1.imp_0a,
                          train1.imp_1a,
                          train1.imp_2a,
                          train1.light_d,
                          train1.light_a,
                          train1.weekday,
                           train1.year,
                          train1.month,
                          train1.passenger_count))
# train_X = np.column_stack((train['haversine'], np.ones(len(train))))
FA = train1['fare_amount'].values
print(train2.shape)
print(FA.shape)

## Normalizatioin

In [ ]:
sigma = train2[:,0:6].std(axis = 0)
mean =train2[:,0:6].mean(axis = 0)

In [ ]:
def normalize(sigma, mean, X):
    X = (X - mean)/sigma
    return X
train2[:,0:6] = normalize(sigma, mean, train2[:,0:6])

## Train Test Split and Train


In [ ]:
#**Check for Random State = 3
from sklearn.model_selection import train_test_split
train_X, val_X, train_Y, val_Y = train_test_split(train2, FA, test_size = 0.001,
                                                 random_state = 3)
print(train_X.shape, val_X.shape, train_Y.shape, val_Y.shape)

#### Error RMSE

In [ ]:
def rmse(pred, actual):
    assert(len(pred) == len(actual))
    return np.sqrt(((pred - actual)**2).sum()/len(pred))

## LightGBM


In [ ]:
lgbmObj = lgbm.LGBMRegressor(n_estimators=2000,
                             learning_rate = 0.03,
                             subsample=0.98,
                             subsample_freq=3,
                             reg_lambda = 0.0,
                             max_depth=7,
                             random_state=42)
lgbmObj.fit(train_X, train_Y,eval_set=[(train_X,train_Y),(val_X,val_Y)],
            eval_metric='rmse',verbose=False)

In [ ]:
print("Training Error = ", rmse(lgbmObj.predict(train_X), train_Y))
print("Validation Error = ", rmse(lgbmObj.predict(val_X), val_Y))

In [ ]:
lgbmObj.feature_importances_

------------------------------------

## Predicting Fare_Amount

In [ ]:
test = pd.read_csv("../input/test.csv")
print(test.shape)

In [ ]:
test.head()

In [ ]:
print(test.columns)

In [ ]:
#Remove null values
test.isnull().sum()

In [ ]:
test['abs_longitude_diff'] = (test['pickup_longitude'] 
                             - test['dropoff_longitude']).abs()
test['abs_latitude_diff'] = (test['pickup_latitude'] 
                            - test['dropoff_latitude']).abs()
haverSine(test)
getDateTime(test)
getYear(test)
getMonth(test)
getTime(test)
getWeekday(test)
getDayOrNight(test)
distFromImpLocations(test)
isHoliday(test)

### Remove Null Values

In [ ]:
test.loc[test['passenger_count'] ==0 ,'passenger_count'
         ]= test['passenger_count'].mode()[0]
test.loc[test['haversine'] == 0, 'haversine'
         ] = test['haversine'].mean()

In [ ]:
#Remove Outliers
print("OldSize = ", test.shape)
test = test[(test.abs_longitude_diff < 5.0) & (test['abs_latitude_diff'] < 5.0)]
print("NewSize = ",test.shape)

In [ ]:
getLightEncoding(test)
test.head()

In [ ]:
#Prepare test data for predictiond
test_X = np.column_stack((test.abs_longitude_diff, 
                          test['abs_longitude_diff']**2,
                          test.abs_latitude_diff,
                          test['abs_latitude_diff']**2,
                          test['haversine'],
                          test['haversine']**2,
                          test.pickup_longitude,
                          test.pickup_latitude,
                          test.dropoff_longitude,
                          test.dropoff_latitude,
                          test.imp_0a,
                          test.imp_1a,
                          test.imp_2a,
                          test.light_d,
                          test.light_a,
                          test.weekday,
                           test.year,
                          test.month,
                          test.passenger_count))
print(test_X.shape)

### Normalization

In [ ]:
test_X[:,:6] = normalize(sigma, mean, test_X[:,:6])

In [ ]:
prediction = lgbmObj.predict(test_X)
sub = pd.DataFrame()
sub['key'] = test.key
sub['fare_amount'] = prediction
sub.to_csv("prediction.csv", index  = False)

In [ ]:
print(sub.shape)
sub.head()